In [ ]:
!pip install pymysql

In [ ]:
!pip install sqlalchemy

In [ ]:
!pip install tqdm

In [ ]:
import csv
from tqdm import tqdm
import pymysql
from sqlalchemy import create_engine, inspect
import pandas as pd
from urllib.parse import quote
import json

from secrets import db_user, db_pass, db_host, db_name

In [ ]:
user = db_user
passw = quote(db_pass)
host = db_host
db = db_name

In [ ]:
connection_str = "mysql+pymysql://{0}:{1}@{2}/{3}" \
    .format(user, passw, host, db)
cnx = create_engine(connection_str)    

In [ ]:

inspector = inspect(cnx)
tables_dict = {}
for table_name in tqdm(inspector.get_table_names()):
    fields_dict = {}
    for column in inspector.get_columns(table_name):
        if "INT" in str(column["type"])\
            or "DECIMAL" in str(column["type"]) \
            or "BOOL" in str(column["type"]):
            quote = False
        else:
            quote = True
        fields_dict[column["name"]] = quote
    tables_dict[table_name] = fields_dict

In [ ]:
#print(json.dumps(tables_dict, indent = 4))

In [ ]:
sales_channel_df = pd.read_sql(
    'SELECT * FROM sales_channel', cnx)

In [ ]:
sales_channel_df.info()

In [ ]:
sales_channel_df.head()

In [ ]:
exceptions_dict = []

def insert_data_into_db(filepath, table):
    table_structure_dict = tables_dict[table]
    full_query = ""
    query_start = "INSERT INTO {0} (".format(table)
    with open(filepath) as f:
        csv_reader = csv.reader(f, delimiter = ',')
        for num, row in enumerate(csv_reader):
            if num == 0:
                query_fields = ",".join(field_name for field_name in row) \
                    + ") "
                quoting_lst = [table_structure_dict[field_name] \
                    for field_name in row] 
                query_values = "VALUES (" \
                    + ",".join(["%s" for field_name in row]) \
                    + ")" 
            else:
                pass  
        query = query_start + query_fields + query_values  
    num = 0
    with open(filepath) as f:
        csv_reader = csv.reader(f, delimiter = ',')    
        for num, row in enumerate(tqdm(csv_reader)):
            if num >= 1:
                values_lst = []
                #query_values = "VALUES (" \
                for value, quote in zip(row, quoting_lst):
                    if value == '' \
                        or value.upper() == 'NONE' \
                        or value.upper() == 'NULL':
                        value_norm = None
                    else:
                        value_norm = "{0}".format(value) 
                    values_lst.append(value_norm)
                try:                   
                    id = cnx.execute(query, values_lst)
                except Exception as e:
                    exception_subdict = {}
                    exception_subdict["EXC"] = e
                    exception_subdict["VALUES"] = values_lst
                    exceptions_dict[num] = "Exc: {0}: {1}".format(e, values_lst)
                    pass
            else:
                pass
    return exceptions_dict
#query = "INSERT INTO  `database_name`.`student` (`name` ,`class` ,`mark` ,`sex`) VALUES(%s,%s,%s,%s)"
#my_data=('King','Five',45,'male')   
#id=my_conn.execute(query,my_data)

In [ ]:
exceptions_dict = insert_data_into_db("/home/sugus/Downloads/articles.csv", "article")

In [102]:
articles_df = pd.read_csv(
    "/home/sugus/Downloads/articles.csv",
    sep = ',', quotechar = '"',
    encoding = 'utf8')

In [104]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [108]:
articles_df.to_sql('article', con = cnx, 
    index = False, if_exists = 'append',
    chunksize = 1000, method = 'multi')